# Phase 3: Template-to-Template Evaluation

This notebook contains results evaluating the correspondence between RheMAP templates (D99, INIA19, macaqueMNI, NMTv1.3, Yerkes19).

In [1]:
# initialize libraries
library(plyr)
library(digest)
library(reshape2)
library(ggplot2)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
# useful functions

# calculate the distance between two sets of coordinates
dist3D <- function(coord1, coord2) { # vector X,Y,Z
        xdist <- coord1[1] - coord2[1] # could also write as coord1$X, etc.
        ydist <- coord1[2] - coord2[2]
        zdist <- coord1[3] - coord2[3]
        euclidean_dist <- sqrt(xdist^2+ydist^2+zdist^2)
        return(as.numeric(unlist(euclidean_dist)))
}

# calculate the pairwise distance between an array of 3D coordinates
pairwise_dist3D <- function(temp_coords) { # labeled X,Y,Z
        N <- length(temp_coords$X)
        dist_vec <- rep(0,N) # create vector
        sum_dist <- 0 # initialize to zero
        count <- 0
        for (i in 1:(N-1)) {
                for (j in (i+1):N) {
                        if (i != j) {
                                count <- count + 1
                                first_coord <- temp_coords[i,]
                                second_coord <- temp_coords[j,]
                                curr_dist <- dist3D(first_coord, second_coord)
                                sum_dist <- sum_dist + curr_dist
                                dist_vec[count] <- curr_dist
                        }
                }
        }
        return(c(as.numeric(unlist(mean(dist_vec))),as.numeric(unlist(sd(dist_vec)))))
}

In [3]:
# initialize variables and load in raw fcsv data into df_raters
setwd('~/Documents/GitHub/afids-macaca/data/PHASE3_RheMAP_transformed/')

df_afid <- read.table('~/Documents/GitHub/afids-macaca/etc/afids.csv', sep=",", header=TRUE)

df_regs <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),
                        moving_template=factor(),fixed_template=factor(),xfm=factor(),
                        name=character(),description=character(),stringsAsFactors = FALSE)
csv_files <- list.files(".", "*.fcsv")

for (i in 1:length(csv_files)) {
    curr_split <- unlist(strsplit(csv_files[i],"_"))
    if (length(curr_split)>1) { # extract name and session data
        moving_template <- curr_split[1]
        fixed_template <- curr_split[5]
        xfm <- as.character(unlist(strsplit(curr_split[6],"[.]"))[1])
    }
    curr_reg <- read.table(csv_files[i], header=FALSE, sep=",")
    df_reg <- data.frame(fid = 1:length(curr_reg$V1))

    df_reg <- cbind(df_reg,X=curr_reg[2],Y=curr_reg[3],Z=curr_reg[4],moving_template=moving_template,
                    fixed_template=fixed_template,
                    xfm=xfm,name=curr_reg[12],
                    description=curr_reg[13])
  
    df_reg <- rename(df_reg, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
    df_regs <- rbind(df_regs,df_reg)
}

levels(df_regs$xfm) <- c('nlin','lin') # rename levels
head(df_regs)


fid,X,Y,Z,moving_template,fixed_template,xfm,name,description
1,-0.269,0.416,0.187,d99,inia19,nlin,1,AC
2,-0.160,-12.484,-1.242,d99,inia19,nlin,2,PC
3,-0.238,-19.890,-3.171,d99,inia19,nlin,3,infracollicular sulcus
4,-0.478,-10.446,-10.850,d99,inia19,nlin,4,PMJ
5,-0.359,-8.024,-6.561,d99,inia19,nlin,5,superior interpeduncular fossa
6,6.844,-13.664,-5.708,d99,inia19,nlin,6,R superior LMS


## Templates to D99

In [4]:
# Start with other templates --> D99

# load D99
df_D99_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),stringsAsFactors = FALSE)
D99_file <- "~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/d99_MEAN.fcsv"

curr_template <- read.table(D99_file, header=FALSE, sep=",")
df_template <- data.frame(fid = 1:length(curr_template$V1))
df_template <- cbind(df_template,X=curr_template[2],Y=curr_template[3],Z=curr_template[4])
df_template <- rename(df_template, c("V2"="X","V3"="Y","V4"="Z"))
df_D99_mean <- df_template

df_D99_results <- subset(df_regs, fixed_template == "d99")
length_df <- length(df_D99_results$fid)
df_D99_results$dist <- rep(0, length_df)

for (val in 1:(length_df-1))
{
    curr_coord <- df_D99_results[val, 2:4]
    curr_fid <- df_D99_results[val,]$fid
    truth_coord <- df_D99_mean[as.numeric(curr_fid),2:4]
    df_D99_results[val,]$dist <- dist3D(curr_coord, truth_coord)
}
#ddply(df_D99_results, .(moving_template,xfm), summarize, mean_dist = mean(dist), sd_dist = sd(dist))


## Templates to inia19

In [5]:
# other templates --> inia19

# load inia19
df_inia19_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),stringsAsFactors = FALSE)
inia19_file <- "~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/inia19_MEAN.fcsv"

curr_template <- read.table(inia19_file, header=FALSE, sep=",")
df_template <- data.frame(fid = 1:length(curr_template$V1))
df_template <- cbind(df_template,X=curr_template[2],Y=curr_template[3],Z=curr_template[4])
df_template <- rename(df_template, c("V2"="X","V3"="Y","V4"="Z"))
df_inia19_mean <- df_template

df_inia19_results <- subset(df_regs, fixed_template == "inia19")
length_df <- length(df_inia19_results$fid)
df_inia19_results$dist <- rep(0, length_df)

for (val in 1:(length_df-1))
{
  curr_coord <- df_inia19_results[val, 2:4]
  curr_fid <- df_inia19_results[val,]$fid
  truth_coord <- df_inia19_mean[as.numeric(curr_fid),2:4]
  df_inia19_results[val,]$dist <- dist3D(curr_coord, truth_coord)
}
#ddply(df_inia19_results, .(moving_template,xfm), summarize, mean_dist = mean(dist), sd_dist = sd(dist))

## Templates to macaqueMNI

In [6]:
# Start with other templates --> macaqueMNI

# load macaqueMNI
df_macaqueMNI_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),stringsAsFactors = FALSE)
macaqueMNI_file <- "~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/macaqueMNI_MEAN.fcsv"

curr_template <- read.table(macaqueMNI_file, header=FALSE, sep=",")
df_template <- data.frame(fid = 1:length(curr_template$V1))
df_template <- cbind(df_template,X=curr_template[2],Y=curr_template[3],Z=curr_template[4])
df_template <- rename(df_template, c("V2"="X","V3"="Y","V4"="Z"))
df_macaqueMNI_mean <- df_template

df_macaqueMNI_results <- subset(df_regs, fixed_template == "macaqueMNI")
length_df <- length(df_macaqueMNI_results$fid)
df_macaqueMNI_results$dist <- rep(0, length_df)

for (val in 1:(length_df-1))
{
  curr_coord <- df_macaqueMNI_results[val, 2:4]
  curr_fid <- df_macaqueMNI_results[val,]$fid
  truth_coord <- df_macaqueMNI_mean[as.numeric(curr_fid),2:4]
  df_macaqueMNI_results[val,]$dist <- dist3D(curr_coord, truth_coord)
}
#ddply(df_macaqueMNI_results, .(moving_template,xfm), summarize, mean_dist = mean(dist), sd_dist = sd(dist))

## Templates to NMTv1.3

In [7]:
# Start with other templates --> nmtv1.3

# load nmtv1.3
df_nmtv1.3_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),stringsAsFactors = FALSE)
nmtv1.3_file <- "~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/nmtv1.3_MEAN.fcsv"

curr_template <- read.table(nmtv1.3_file, header=FALSE, sep=",")
df_template <- data.frame(fid = 1:length(curr_template$V1))
df_template <- cbind(df_template,X=curr_template[2],Y=curr_template[3],Z=curr_template[4])
df_template <- rename(df_template, c("V2"="X","V3"="Y","V4"="Z"))
df_nmtv1.3_mean <- df_template

df_nmtv1.3_results <- subset(df_regs, fixed_template == "nmtv1.3")
length_df <- length(df_nmtv1.3_results$fid)
df_nmtv1.3_results$dist <- rep(0, length_df)

for (val in 1:(length_df-1))
{
  curr_coord <- df_nmtv1.3_results[val, 2:4]
  curr_fid <- df_nmtv1.3_results[val,]$fid
  truth_coord <- df_nmtv1.3_mean[as.numeric(curr_fid),2:4]
  df_nmtv1.3_results[val,]$dist <- dist3D(curr_coord, truth_coord)
}
#ddply(df_nmtv1.3_results, .(moving_template,xfm), summarize, mean_dist = mean(dist), sd_dist = sd(dist))

## Templates to Yerkes19

In [8]:
# Start with other templates --> yerkes19

# load yerkes19
df_yerkes19_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),stringsAsFactors = FALSE)
yerkes19_file <- "~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/yerkes19_MEAN.fcsv"

curr_template <- read.table(yerkes19_file, header=FALSE, sep=",")
df_template <- data.frame(fid = 1:length(curr_template$V1))
df_template <- cbind(df_template,X=curr_template[2],Y=curr_template[3],Z=curr_template[4])
df_template <- rename(df_template, c("V2"="X","V3"="Y","V4"="Z"))
df_yerkes19_mean <- df_template

df_yerkes19_results <- subset(df_regs, fixed_template == "yerkes19")
length_df <- length(df_yerkes19_results$fid)
df_yerkes19_results$dist <- rep(0, length_df)

for (val in 1:(length_df-1))
{
  curr_coord <- df_yerkes19_results[val, 2:4]
  curr_fid <- df_yerkes19_results[val,]$fid
  truth_coord <- df_yerkes19_mean[as.numeric(curr_fid),2:4]
  df_yerkes19_results[val,]$dist <- dist3D(curr_coord, truth_coord)
}
#ddply(df_yerkes19_results, .(moving_template,xfm), summarize, mean_dist = mean(dist), sd_dist = sd(dist))

In [9]:
# Merge them all back together
df_all <- rbind(df_D99_results, df_inia19_results)
df_all <- rbind(df_all, df_macaqueMNI_results)
df_all <- rbind(df_all, df_nmtv1.3_results)
df_all <- rbind(df_all, df_yerkes19_results)

In [10]:
# assessment of accuracy for moving template
ddply(df_all, .(moving_template,xfm), summarize, mean_dist = mean(dist), sd_dist = sd(dist))

moving_template,xfm,mean_dist,sd_dist
d99,nlin,0.6701923,0.4727266
d99,lin,1.0734210,0.5721741
inia19,nlin,0.7206973,0.5626851
inia19,lin,0.9380699,0.6053298
macaqueMNI,nlin,0.5177407,0.3605860
macaqueMNI,lin,0.8667511,0.5135491
nmtv1.3,nlin,0.8415324,0.9151837
nmtv1.3,lin,0.9708099,0.6926570
yerkes19,nlin,0.7146474,0.5276831
yerkes19,lin,1.2838677,0.5281260


In [11]:
# assessment of accuracy for fixed template
ddply(df_all, .(fixed_template,xfm), summarize, mean_dist = mean(dist), sd_dist = sd(dist))

fixed_template,xfm,mean_dist,sd_dist
inia19,nlin,0.7538142,0.6028571
inia19,lin,0.9286076,0.6049572
macaqueMNI,nlin,0.4904819,0.3421030
macaqueMNI,lin,0.7835461,0.4637025
nmtv1.3,nlin,0.7109190,0.6738451
nmtv1.3,lin,0.9323473,0.6556214
yerkes19,nlin,0.8304200,0.8053213
yerkes19,lin,1.4079595,0.5092159
d99,nlin,0.6791750,0.4397577
d99,lin,1.0804591,0.5693225


In [12]:
# assessment of afids after nonlinear registration
subset(ddply(subset(df_all, xfm == 'nlin'), .(fid,description), summarize, mean_dist = mean(dist), sd_dist = sd(dist)), mean_dist > 1)

,fid,description,mean_dist,sd_dist
10,10,culmen,1.157730,0.8131759
25,25,R inferior AM temporal horn,1.332857,0.9094108
26,26,L inferior AM temporal horn,1.207916,0.8075759
29,29,R ventral occipital horn,1.462895,1.0851031
30,30,L ventral occipital horn,1.292283,0.8382223


In [21]:
# check for max values (highlights areas prone to misregistration)
#ddply(subset(df_all, xfm == 'nlin' & dist > 1.5), .(fid,description), summarize, mean_dist = mean(dist))
subset(df_all, xfm == 'nlin' & dist > 1.5)

,fid,X,Y,Z,moving_template,fixed_template,xfm,name,description,dist
266,10,-0.196,-29.460,6.937,inia19,d99,nlin,10,culmen,1.938805
281,25,9.826,-2.571,-13.172,inia19,d99,nlin,25,R inferior AM temporal horn,1.953989
797,29,16.406,-26.860,1.900,nmtv1.3,d99,nlin,29,R ventral occipital horn,2.274276
798,30,-16.594,-26.301,2.062,nmtv1.3,d99,nlin,30,L ventral occipital horn,1.897893
1049,25,8.764,-4.246,-12.782,yerkes19,d99,nlin,25,R inferior AM temporal horn,2.045772
1050,26,-9.894,-3.898,-13.270,yerkes19,d99,nlin,26,L inferior AM temporal horn,2.001288
4,4,-0.478,-10.446,-10.850,d99,inia19,nlin,4,PMJ,1.544082
10,10,-0.185,-28.002,4.848,d99,inia19,nlin,10,culmen,2.285191
25,25,8.399,-2.061,-14.082,d99,inia19,nlin,25,R inferior AM temporal horn,2.364057
586,10,0.089,-28.720,3.898,macaqueMNI,inia19,nlin,10,culmen,1.632390


In [13]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Catalina 10.15.4

Matrix products: default
BLAS/LAPACK: /Users/jclau/anaconda3/envs/r-tutorial/lib/R/lib/libRblas.dylib

locale:
[1] en_CA.UTF-8/UTF-8/en_CA.UTF-8/C/en_CA.UTF-8/en_CA.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] ggplot2_3.1.1  reshape2_1.4.3 digest_0.6.18  plyr_1.8.4    

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1       magrittr_1.5     tidyselect_0.2.5 munsell_0.5.0   
 [5] uuid_0.1-2       colorspace_1.4-1 R6_2.4.0         rlang_0.3.4     
 [9] dplyr_0.8.0.1    stringr_1.4.0    tools_3.6.1      grid_3.6.1      
[13] gtable_0.3.0     withr_2.1.2      htmltools_0.3.6  assertthat_0.2.1
[17] lazyeval_0.2.2   tibble_2.1.1     crayon_1.3.4     IRdisplay_0.7.0 
[21] purrr_0.3.2      repr_0.19.2      base64enc_0.1-3  IRkernel_0.8.15 
[25] glue_1.3.1       evaluate_0.13    pbdZMQ